In [ ]:
import csv
import numpy as np
import pandas as pd

def split_dataset():
    string_judgment = 'judgment'
    string_dataset = "dataset"

    df_judgments = pd.read_csv('judgements_all.csv', delimiter=',', quoting =csv.QUOTE_MINIMAL)
    # df_uses = pd.read_csv('uses_all.csv', delimiter='\t', quoting =3)
    # print(len(df_judgments))
    # print(len(df_uses))

    print("column headers for dataframe: ")
    print(df_judgments.keys())

    dataset_list = df_judgments[string_dataset].unique().tolist()

    for dataset_name in dataset_list:
        unique_labels_of_dataset = df_judgments[df_judgments[string_dataset] == dataset_name][string_judgment].unique().tolist()
        if np.nan in unique_labels_of_dataset:
            print(dataset_name + " has nan value")
        # print(dataset_name + str(unique_labels_of_dataset))

    df_aggregated = df_judgments[df_judgments[string_dataset].isin(['WIC', 'Cosimlex', "TempoWic"])]

    df_judgments = df_judgments[~df_judgments[string_dataset].isin(['WIC', 'Cosimlex', "TempoWic"])]


    # df_judgements_with_vote_five = df_judgments[df_judgments[key_for_vote] == 5]
    # print(df_judgements_with_vote_five["dataset"].unique())

    # df_usim = df_judgments[df_judgments['dataset'] == "USim"]
    # df_judgments = df_judgments[df_judgments['dataset'] != "USim"]
    # print(df_judgments[df_judgments['dataset'] == "USim"]["judgment"].unique())


    # df_aggregated[key_for_vote].replace({'T': 1, 'F': 0}, inplace=True)

    row_index_of_null_vote = df_judgments.loc[df_judgments[string_judgment].isnull()].index.tolist()
    print("number of rows with nan judgment in dataset rushifteval_public: " + str(len(row_index_of_null_vote)))

    # csv_row_index_null_vote = [index + 2 for index in df_judgements.loc[df_judgements[key_for_vote].isnull()].index]
    # print(df_judgements.iloc[csv_row_index_null_vote[0:5]])
    # print(f"The null value in column 'judgment' is located on csv row {csv_row_index_null_vote}.")

    complement =  ~df_judgments.index.isin(row_index_of_null_vote)

    df_judgments[string_judgment][complement] = df_judgments[string_judgment][complement].astype(float)

    df_judgments[string_judgment].replace({0.0: np.nan}, inplace=True)

    # print(df_judgments[string_dataset].unique())

    # print(df_judgments[string_judgment].unique())
    # print(df_usim["judgment"].unique())

    # print(df_aggregated[string_judgment].unique())

    return df_judgments, df_aggregated

df_judgments, df_aggregated = split_dataset()
print(df_aggregated.head())
print(df_judgments.head())

In [ ]:
def get_instances_with_multiple_annotations(df_judgments):
    key_identifier_one = 'identifier1'
    key_identifier_two = 'identifier2'
    key_dataset = 'dataset'
    key_judgment = "judgment"
    dict_of_nonaggregated_instances = dict()
    for index, row in df_judgments.iterrows():
        composite_key = (row[key_dataset], row[key_identifier_one], row[key_identifier_two])
        if composite_key not in dict_of_nonaggregated_instances:
            dict_of_nonaggregated_instances[composite_key] = [tuple(list(composite_key) + [index, row[key_judgment]])]
        else:
            dict_of_nonaggregated_instances[composite_key] = dict_of_nonaggregated_instances[composite_key] + [tuple(list(composite_key) + [index, row[key_judgment]])]

    dict_of_nonaggregated_instances_with_multiple_annotations = dict()
    row_count = 0
    for key, value in dict_of_nonaggregated_instances.items():
        if len(value) != 1:
            dict_of_nonaggregated_instances_with_multiple_annotations[key] = value
            row_count = row_count + len(value)
        else:
            row_count = row_count + 1
    print("row_count: " + str(row_count))
    return dict_of_nonaggregated_instances_with_multiple_annotations

def get_dict_of_nonaggregated_instances(df_judgments):
    key_identifier_one = 'identifier1'
    key_identifier_two = 'identifier2'
    key_dataset = 'dataset'
    key_judgment = "judgment"
    dict_of_nonaggregated_instances = dict()
    for index, row in df_judgments.iterrows():
        composite_key = (row[key_dataset], row[key_identifier_one], row[key_identifier_two])
        if composite_key not in dict_of_nonaggregated_instances:
            dict_of_nonaggregated_instances[composite_key] = [tuple(list(composite_key) + [index, row[key_judgment]])]
        else:
            dict_of_nonaggregated_instances[composite_key] = dict_of_nonaggregated_instances[composite_key] + [tuple(list(composite_key) + [index, row[key_judgment]])]

    return dict_of_nonaggregated_instances

def get_rows_of_instances_with_multiple_annotations(df_judgments):
    rows_of_nonaggregated_instances_with_multiple_annotations = list()
    key_identifier_one = 'identifier1'
    key_identifier_two = 'identifier2'
    key_dataset = 'dataset'
    key_judgment = "judgment"
    dict_of_nonaggregated_instances = dict()
    for index, row in df_judgments.iterrows():
        composite_key = (row[key_dataset], row[key_identifier_one], row[key_identifier_two])
        if composite_key not in dict_of_nonaggregated_instances:
            dict_of_nonaggregated_instances[composite_key] = [tuple(list(composite_key) + [index, row[key_judgment]])]
        else:
            dict_of_nonaggregated_instances[composite_key] = dict_of_nonaggregated_instances[composite_key] + [tuple(list(composite_key) + [index, row[key_judgment]])]

    row_count = 0
    for key, value in dict_of_nonaggregated_instances.items():
        if len(value) != 1:
            for tuple_annotation in value:
                rows_of_nonaggregated_instances_with_multiple_annotations.append(tuple_annotation[3])
            row_count = row_count + len(value)
        else:
            row_count = row_count + 1
    print("row_count: " + str(row_count))
    return rows_of_nonaggregated_instances_with_multiple_annotations

# rows_of_nonaggregated_instances_with_multiple_annotations = get_rows_of_instances_with_multiple_annotations(df_judgments)
# print(len(rows_of_nonaggregated_instances_with_multiple_annotations))

# df_judgments_removing_instances_of_with_multiple_annotations = df_judgments.drop(rows_of_nonaggregated_instances_with_multiple_annotations)

# print(len(get_rows_of_instances_with_multiple_annotations(df_judgments_removing_instances_of_with_multiple_annotations)))

def get_votes_for_instances_with_multiple_annotations(df_judgments):
    dict_of_nonaggregated_instances_with_multiple_annotations = get_instances_with_multiple_annotations(df_judgments)
    for key, value in dict_of_nonaggregated_instances_with_multiple_annotations.items():
        votes_list = list()
        for tuple_annotation in value:
            votes_list.append(tuple_annotation[-1])
        votes_list.sort()
        dict_of_nonaggregated_instances_with_multiple_annotations[key] = votes_list
    return dict_of_nonaggregated_instances_with_multiple_annotations

# print(get_instances_with_multiple_annotations(df_judgments.head(2000)))
# print(get_votes_for_instances_with_multiple_annotations(df_judgments.head(2000)))

def get_votes_for_nonaggregated_instances(df_judgments):
    dict_of_nonaggregated_instances = get_dict_of_nonaggregated_instances(df_judgments)
    for key, value in dict_of_nonaggregated_instances.items():
        votes_list = list()
        for tuple_annotation in value:
            votes_list.append(tuple_annotation[-1])
        votes_list.sort()
        dict_of_nonaggregated_instances[key] = votes_list
    return dict_of_nonaggregated_instances

In [ ]:
def aggregation_median_rounding(dict_of_nonaggregated_instances_with_multiple_annotations: dict):
    for key, value in dict_of_nonaggregated_instances_with_multiple_annotations.items():
        array = np.array(value)
        median = np.nanmedian(array)
        if not np.isnan(median):
            dict_of_nonaggregated_instances_with_multiple_annotations[key] = float(round(median))
        else:
            dict_of_nonaggregated_instances_with_multiple_annotations[key] = np.nan
    return dict_of_nonaggregated_instances_with_multiple_annotations

# print(aggregation_median_rounding(get_votes_for_instances_with_multiple_annotations(df_judgments.head(2000))))

def aggregation_median_exclusion(dict_of_nonaggregated_instances_with_multiple_annotations: dict):
    for key, value in dict_of_nonaggregated_instances_with_multiple_annotations.items():
        array = np.array(value)
        median = np.nanmedian(array)
        if median.is_integer():
            dict_of_nonaggregated_instances_with_multiple_annotations[key] = median
        else:   
            dict_of_nonaggregated_instances_with_multiple_annotations[key] = np.nan
    return dict_of_nonaggregated_instances_with_multiple_annotations

# print(aggregation_median_exclusion(get_votes_for_instances_with_multiple_annotations(df_judgments.head(2000))))


In [ ]:

def cleaning_remove_nan(dict_of_nonaggregated_instances: dict):
    for key, value in dict_of_nonaggregated_instances.items():
        if (np.nan in value):
            dict_of_nonaggregated_instances[key] = [np.nan for _ in range(len(value))]
    return dict_of_nonaggregated_instances

def cleaning_remove_any_disagreements(dict_of_nonaggregated_instances: dict):
    dict_of_nonaggregated_instances = cleaning_remove_nan(dict_of_nonaggregated_instances)
    for key, value in dict_of_nonaggregated_instances.items():
        unique_labels_list = list(set(value))
        if len(unique_labels_list) > 1:
            dict_of_nonaggregated_instances[key] = [np.nan for _ in range(len(value))]
    return dict_of_nonaggregated_instances
    
def cleaning_allow_disagreements_in_one_point(dict_of_nonaggregated_instances: dict):
    dict_of_nonaggregated_instances = cleaning_remove_nan(dict_of_nonaggregated_instances)
    for key, value in dict_of_nonaggregated_instances.items():
        unique_labels_list = list(set(value))
        if len(unique_labels_list) > 2:
            dict_of_nonaggregated_instances[key] = [np.nan for _ in range(len(value))]
        if (len(unique_labels_list) == 2):
            if abs(unique_labels_list[0] - unique_labels_list[1]) > 1:
                dict_of_nonaggregated_instances[key] = [np.nan for _ in range(len(value))]
    return dict_of_nonaggregated_instances

def cleaning_remove_instance_with_one_valid_vote(dict_of_nonaggregated_instances: dict):
    for key, value in dict_of_nonaggregated_instances.items():
        unique_labels_list = list(set(value))
        if len(unique_labels_list) == 1:
            dict_of_nonaggregated_instances[key] = [np.nan]
    return dict_of_nonaggregated_instances

def cleaning_most_strict_condition(dict_of_nonaggregated_instances: dict):
    for key, value in dict_of_nonaggregated_instances.items():
        unique_labels_list = list(set(value))
        if (np.nan in value):
            dict_of_nonaggregated_instances[key] = [np.nan for _ in range(len(value))]
        if len(unique_labels_list) > 1:
            dict_of_nonaggregated_instances[key] = [np.nan for _ in range(len(value))]
        if len(value) == 1:
            dict_of_nonaggregated_instances[key] = [np.nan]
    return dict_of_nonaggregated_instances

def cleaning_second_strict_condition(dict_of_nonaggregated_instances: dict):
    for key, value in dict_of_nonaggregated_instances.items():
        unique_labels_list = list(set(value))
        if (np.nan in value):
            dict_of_nonaggregated_instances[key] = [np.nan for _ in range(len(value))]
        if len(unique_labels_list) > 2:
            dict_of_nonaggregated_instances[key] = [np.nan for _ in range(len(value))]
        if (len(unique_labels_list) == 2):
            if abs(unique_labels_list[0] - unique_labels_list[1]) > 1:
                dict_of_nonaggregated_instances[key] = [np.nan for _ in range(len(value))]
        if len(unique_labels_list) == 1:
            dict_of_nonaggregated_instances[key] = [np.nan]
    return dict_of_nonaggregated_instances

# print(cleaning_remove_any_disagreements(get_votes_for_instances_with_multiple_annotations(df_judgments.head(2000))))
# print(cleaning_allow_disagreements_in_one_point(get_votes_for_instances_with_multiple_annotations(df_judgments.head(2000))))

In [ ]:
df_judgments, _ = split_dataset()
dict_of_nonaggregated_instances = get_votes_for_nonaggregated_instances(df_judgments)
dict_of_nonaggregated_instances = cleaning_most_strict_condition(dict_of_nonaggregated_instances)


count = 0
instance = 0
for key, value in dict_of_nonaggregated_instances.items():
    instance = instance + 1
    # print(instance)
    if np.nan not in value:
        if key[0]!= 'RuSemShift':
            print(key, value)
        count = count + 1
print(count)

In [ ]:
df_judgments, _ = split_dataset()
df_dwug_de = df_judgments[df_judgments["dataset"] == "dwug_de"]
print(df_dwug_de["judgment"].value_counts())